# Checkpointing in Workflows

During the execution of a Workflow, `Checkpoint` objects are automatically stored within the `Context` class. By default, `Checkpoints` are automatically created at the completion of a step. At such instants in the Workflow execution, we checkpoint the name of the last completed step, it's input `Event`, it's output `Event`, as well as a snapshot of the `Context` itself. We also store a `Checkpoint` before the emission of the `StartEvent`, but note that running from these special checkpoints are essentially equivalent to the typical execution of a `Workflow` -- we add it here purely for convenience.

In this notebook, we demonstrate how to make use of such checkpoints. Specifically, we'll show how to obtain the list of all of the stored checkpoints, filter this list, and even how to re-run a `Workflow` from any one them.

## Defining the Workflow

In [ ]:
import os

api_key = os.environ.get("OPENAI_API_KEY")

In [ ]:
from llama_index.core.workflow import (
    Workflow,
    step,
    StartEvent,
    StopEvent,
    Event,
    Context,
)
from llama_index.llms.openai import OpenAI


class JokeEvent(Event):
    joke: str


class JokeFlow(Workflow):
    llm = OpenAI(api_key=api_key)

    @step
    async def generate_joke(self, ev: StartEvent) -> JokeEvent:
        topic = ev.topic

        prompt = f"Write your best joke about {topic}."
        response = await self.llm.acomplete(prompt)
        return JokeEvent(joke=str(response))

    @step
    async def critique_joke(self, ev: JokeEvent) -> StopEvent:
        joke = ev.joke

        prompt = f"Give a thorough analysis and critique of the following joke: {joke}"
        response = await self.llm.acomplete(prompt)
        return StopEvent(result=str(response))

### Running the Workflow

In [ ]:
# instantiate Jokeflow
workflow = JokeFlow()
ctx = Context(workflow=workflow)

# run it
handler = workflow.run(ctx=ctx, topic="math")
await handler

'Analysis:\nThis joke plays on the mathematical concept of the equal sign, which is used to show that two quantities are the same. The humor comes from personifying the equal sign and attributing human emotions and characteristics to it. The joke relies on the double meaning of the word "equal" - not only in the mathematical sense but also in the sense of being humble and not thinking oneself better or worse than others.\n\nCritique:\nThis joke is clever and plays on a common mathematical symbol in a humorous way. The punchline is unexpected and plays on the idea of humility, which adds an extra layer to the joke. However, the joke may not be immediately understood by everyone, especially those who are not familiar with mathematical symbols. Additionally, the humor may be considered somewhat niche and may not appeal to a wide audience. Overall, while the joke is clever and well-crafted, its niche appeal may limit its effectiveness in reaching a broader audience.'

### Listing the saved Checkpoints

In [ ]:
handler.ctx.checkpoints

[Checkpoint(id_='ed3ba540-b174-4d97-add8-87fce11dba48', last_completed_step=None, input_event=None, output_event=StartEvent(), ctx_state={'globals': {}, 'streaming_queue': '[]', 'queues': {'_done': '[]', 'critique_joke': '[]', 'generate_joke': '[]'}, 'stepwise': False, 'events_buffer': {}, 'accepted_events': [('generate_joke', 'StartEvent'), ('critique_joke', 'JokeEvent')], 'broker_log': [], 'is_running': False}),
 Checkpoint(id_='d65bd3e1-38f6-44b1-90fe-d024cd14dead', last_completed_step='generate_joke', input_event=StartEvent(), output_event=JokeEvent(joke="Why was the equal sign so humble?\n\nBecause he knew he wasn't less than or greater than anyone else."), ctx_state={'globals': {}, 'streaming_queue': '[]', 'queues': {'_done': '[]', 'critique_joke': '[]', 'generate_joke': '[]'}, 'stepwise': False, 'events_buffer': {}, 'accepted_events': [('generate_joke', 'StartEvent'), ('critique_joke', 'JokeEvent')], 'broker_log': ['{"__is_pydantic": true, "value": {"id_": "ed3ba540-b174-4d97-ad

In [ ]:
len(handler.ctx.checkpoints)

3

There are 3 stored checkpoints after this first run. The first of these checkpoints are created just before the emission of the `StartEvent`. This can be thought of as a "startup" step being completed which has a null input `Event` and whose output event is the `StartEvent`. The remaining two of these checkpoints are made at the completion of each of the two steps in this Workflow. 

### Filtering the Checkpoints

Before showcasing how to filter through checkpoints, let's first make things a bit more interesting by executing the workflow a few more times. Here, we'll run the workflow for two more additional topics. Since each entire run will result in 3 stored checkpoints each, we should have a total of 9 checkpoints after executing the below cell. 

In [ ]:
additional_topics = ["biology", "history"]

for topic in additional_topics:
    handler = workflow.run(
        ctx=ctx, topic=topic
    )  # same Context that was used before
    await handler

len(handler.ctx.checkpoints)

9

In [ ]:
# Filter by output event StopEvent
checkpoints_that_emit_stop_event = ctx.filter_checkpoints(
    output_event_type=StopEvent
)
len(checkpoints_that_emit_stop_event)
[ckpt.output_event.result[:150] for ckpt in checkpoints_that_emit_stop_event]

['Analysis:\nThis joke plays on the mathematical concept of the equal sign, which is used to show that two quantities are the same. The humor comes from ',
 'This joke plays on the double meaning of the word "cell" - in biology, a cell is the basic structural and functional unit of all living organisms, whi',
 'Analysis:\nThis joke plays on the double meaning of the word "pupils," which can refer to both the students in a classroom and the black part of the ey']

In [ ]:
# Filter by the name of last completed step
checkpoints_right_after_generate_joke_step = ctx.filter_checkpoints(
    last_completed_step="generate_joke"
)
len(checkpoints_right_after_generate_joke_step)
[
    ckpt.output_event.joke[:150]
    for ckpt in checkpoints_right_after_generate_joke_step
]

["Why was the equal sign so humble?\n\nBecause he knew he wasn't less than or greater than anyone else.",
 'Why did the biologist break up with the mathematician?\n\nBecause they couldn\'t find a common "cell"!',
 "Why did the history teacher go to jail?\n\nBecause he couldn't control his pupils!"]

### Re-Run Workflow from a specific checkpoint

In [ ]:
ckpt = checkpoints_right_after_generate_joke_step[1]
handler = workflow.run_from(checkpoint=ckpt, ctx=ctx)
await handler

'This joke plays on the double meaning of the word "cell" - in biology, a cell is the basic structural and functional unit of all living organisms, while in mathematics, a cell can refer to a unit in a grid or matrix. The joke implies that the biologist and mathematician couldn\'t find a common ground or connection because they were using the word "cell" in different contexts.\n\nOverall, the joke is clever and plays on the different disciplines of biology and mathematics. It relies on a pun to create humor, which can be effective in engaging the audience. However, some may find the joke to be a bit simplistic or predictable, as it follows a common formula of setting up a scenario with two different types of people and revealing a pun as the punchline.\n\nIn terms of critique, the joke could be seen as reinforcing stereotypes about the differences between scientists and mathematicians. It may also be considered niche humor, as it requires a basic understanding of biology and mathematic

Since we've executed from the checkpoint that represents the end of "generate_joke" step, there is only one additional checkpoint that gets stored i.e., the one that is stored after the completion of the next step, namely "critique_joke".

In [ ]:
len(ctx.checkpoints)

10